In [1]:
library(tidyverse)
library(stringr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


# NBA shot range analysis

## Raw data

To begin, I retrieved raw data for game logs for each team from <https://stats.nba.com/stats/teamgamelog/?TeamID=&Season=2018-19&SeasonType=Regular%20Season> as well as shot chart data for each player from <https://stats.nba.com/stats/shotchartdetail?Period=0&VsConference=&LeagueID=00&LastNGames=0&TeamID=0&Position=&Location=&Outcome=&ContextMeasure=FGA&DateFrom=&StartPeriod=&DateTo=&OpponentTeamID=0&ContextFilter=&RangeType=&Season=2018-19&AheadBehind=&PlayerID={}&EndRange=&VsDivision=&PointDiff=&RookieYear=&GameSegment=&Month=0&ClutchTime=&StartRange=&EndPeriod=&SeasonType=Regular+Season&SeasonSegment=&GameID=&PlayerPosition=>. In the URLs from above we must a TeamID from <https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation#current-teams> or any PlayerID from the league to get the raw JSON data. I saved the data in a csv to be able to analyze later.


### Raw shot chart data

In [2]:
shots <- read.csv("shotcharts-2018-19.csv")

                         
allShotShort <- shots %>% select(GAME_ID, TEAM_ID,TEAM_NAME,PERIOD, ACTION_TYPE,MINUTES_REMAINING, SECONDS_REMAINING, PLAYER_ID, PLAYER_NAME, EVENT_TYPE, SHOT_TYPE, SHOT_ZONE_BASIC, SHOT_ZONE_AREA, SHOT_ZONE_RANGE,
                                 SHOT_ATTEMPTED_FLAG, SHOT_MADE_FLAG,SHOT_DISTANCE, LOC_X, LOC_Y, HTM, VTM)

head(allShotShort)

GAME_ID,TEAM_ID,TEAM_NAME,PERIOD,ACTION_TYPE,MINUTES_REMAINING,SECONDS_REMAINING,PLAYER_ID,PLAYER_NAME,EVENT_TYPE,⋯,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,SHOT_DISTANCE,LOC_X,LOC_Y,HTM,VTM
21800001,1610612738,Boston Celtics,1,Driving Layup Shot,3,47,201143,Al Horford,Made Shot,⋯,Restricted Area,Center(C),Less Than 8 ft.,1,1,2,24,-1,BOS,PHI
21800001,1610612738,Boston Celtics,2,Alley Oop Layup shot,5,7,201143,Al Horford,Made Shot,⋯,Restricted Area,Center(C),Less Than 8 ft.,1,1,1,-2,11,BOS,PHI
21800001,1610612738,Boston Celtics,2,Driving Finger Roll Layup Shot,2,24,201143,Al Horford,Made Shot,⋯,Restricted Area,Center(C),Less Than 8 ft.,1,1,1,-9,7,BOS,PHI
21800001,1610612738,Boston Celtics,3,Jump Shot,11,5,201143,Al Horford,Made Shot,⋯,Mid-Range,Left Side(L),8-16 ft.,1,1,12,-114,53,BOS,PHI
21800001,1610612738,Boston Celtics,4,Fadeaway Jump Shot,7,24,201143,Al Horford,Missed Shot,⋯,In The Paint (Non-RA),Center(C),Less Than 8 ft.,1,0,6,40,57,BOS,PHI
21800001,1610612738,Boston Celtics,4,Jump Shot,4,31,201143,Al Horford,Missed Shot,⋯,Right Corner 3,Right Side(R),24+ ft.,1,0,24,236,53,BOS,PHI


### Raw Game Logs

In [3]:
logs <- read.csv('all-game-logs.csv')
head(logs)

X,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,⋯,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612737,21800472,"DEC 21, 2018",ATL @ NYK,W,8,23,0.258,240,⋯,0.750,15,39,54,29,4,1,16,18,114
1,1610612737,21800452,"DEC 18, 2018",ATL vs. WAS,W,7,23,0.233,240,⋯,0.821,13,36,49,23,4,5,23,29,118
2,1610612737,21800436,"DEC 16, 2018",ATL @ BKN,L,6,23,0.207,240,⋯,0.760,8,25,33,38,9,5,14,22,127
3,1610612737,21800420,"DEC 14, 2018",ATL @ BOS,L,6,22,0.214,240,⋯,0.625,16,28,44,27,10,5,24,19,108
4,1610612737,21800412,"DEC 12, 2018",ATL @ DAL,L,6,21,0.222,240,⋯,0.714,16,28,44,21,7,2,21,30,107
5,1610612737,21800380,"DEC 08, 2018",ATL vs. DEN,W,6,20,0.231,240,⋯,0.688,9,42,51,33,9,11,18,23,106


## Data Wrangling

There was a log of data wrangling and summarizing involved. To start I need to merge the shot chart data with the game log data and select the necessary columns.

In [4]:
alllogs <- merge(allShotShort,logs,by.x= c("GAME_ID", "TEAM_ID"),by.y=c("Game_ID","Team_ID"))


allLogShort <- alllogs %>% select(GAME_ID, TEAM_ID,TEAM_NAME,PERIOD, ACTION_TYPE,MINUTES_REMAINING, SECONDS_REMAINING, PLAYER_ID, PLAYER_NAME,   
                                  EVENT_TYPE, SHOT_TYPE, SHOT_ZONE_BASIC, SHOT_ZONE_AREA, SHOT_ZONE_RANGE, 
                                  SHOT_ATTEMPTED_FLAG, SHOT_MADE_FLAG,SHOT_DISTANCE, LOC_X, LOC_Y, HTM, VTM,GAME_DATE, MATCHUP, WL)


Next there were some initial variables that I wanted so I created a variable for location (home or away), the opponent, a concatenation of date and opponent, team abbreviations, and finally the shot range.

In [5]:
allLogShort$HOMEAWAY <- "Away"
allLogShort$HOMEAWAY[str_detect(allLogShort$MATCHUP, "vs") ]<- "Home"

allLogShort$OPPONENT <-str_sub(allLogShort$MATCHUP,-3)

allLogShort$DATE_OPPONENT <- paste(allLogShort$GAME_DATE, str_sub(allLogShort$MATCHUP,5), sep = " ")

allLogShort$TEAM_ABRV <- substr(allLogShort$MATCHUP, 0, 3)

allLogShort <- allLogShort %>% mutate(RANGE = ifelse(SHOT_DISTANCE <= 4, "RIM",
                                                     ifelse(SHOT_DISTANCE > 4 & SHOT_DISTANCE < 14, "SMR", 
                                                            ifelse(SHOT_DISTANCE > 14 & SHOT_TYPE == "2PT Field Goal", "LMR", 
                                                            ifelse(SHOT_DISTANCE >= 22 & SHOT_TYPE == "3PT Field Goal", "3PT", "LMR")))))
allLogShort <- allLogShort %>% mutate(SHOT_VALUE = ifelse(SHOT_TYPE == "3PT Field Goal", 3,2))

head(allLogShort)

GAME_ID,TEAM_ID,TEAM_NAME,PERIOD,ACTION_TYPE,MINUTES_REMAINING,SECONDS_REMAINING,PLAYER_ID,PLAYER_NAME,EVENT_TYPE,⋯,VTM,GAME_DATE,MATCHUP,WL,HOMEAWAY,OPPONENT,DATE_OPPONENT,TEAM_ABRV,RANGE,SHOT_VALUE
21800001,1610612738,Boston Celtics,3,Tip Layup Shot,0,45,202694,Marcus Morris,Made Shot,⋯,PHI,"OCT 16, 2018",BOS vs. PHI,W,Home,PHI,"OCT 16, 2018 vs. PHI",BOS,RIM,2
21800001,1610612738,Boston Celtics,2,Driving Finger Roll Layup Shot,2,24,201143,Al Horford,Made Shot,⋯,PHI,"OCT 16, 2018",BOS vs. PHI,W,Home,PHI,"OCT 16, 2018 vs. PHI",BOS,RIM,2
21800001,1610612738,Boston Celtics,2,Pullup Jump shot,10,41,202330,Gordon Hayward,Missed Shot,⋯,PHI,"OCT 16, 2018",BOS vs. PHI,W,Home,PHI,"OCT 16, 2018 vs. PHI",BOS,SMR,2
21800001,1610612738,Boston Celtics,3,Jump Shot,4,39,1626179,Terry Rozier,Missed Shot,⋯,PHI,"OCT 16, 2018",BOS vs. PHI,W,Home,PHI,"OCT 16, 2018 vs. PHI",BOS,3PT,3
21800001,1610612738,Boston Celtics,4,Pullup Jump shot,4,5,1628369,Jayson Tatum,Made Shot,⋯,PHI,"OCT 16, 2018",BOS vs. PHI,W,Home,PHI,"OCT 16, 2018 vs. PHI",BOS,LMR,2
21800001,1610612738,Boston Celtics,2,Pullup Jump shot,11,50,202330,Gordon Hayward,Made Shot,⋯,PHI,"OCT 16, 2018",BOS vs. PHI,W,Home,PHI,"OCT 16, 2018 vs. PHI",BOS,SMR,2


Next, I needed a bunch of summary statistics for my league and team analysis. I needed team and player totals but also team and player totals by shot range. Both are shown below.

In [6]:
playerStatsByRange <- allLogShort %>% group_by(PLAYER_NAME, RANGE) %>% summarise(FGM = sum(SHOT_MADE_FLAG),FGA = sum(SHOT_ATTEMPTED_FLAG),
                                                                         FG_PERCENTAGE = FGM/FGA, PPS =sum(SHOT_MADE_FLAG * SHOT_VALUE/FGA),  PTS  = sum(SHOT_MADE_FLAG * SHOT_VALUE))
playerStats <- allLogShort %>% group_by(PLAYER_NAME) %>% summarise(FGM = sum(SHOT_MADE_FLAG),FGA = sum(SHOT_ATTEMPTED_FLAG),
                                                           PTS  = sum(SHOT_MADE_FLAG * SHOT_VALUE), EFG = PTS/(FGA*2), PPS = PTS/FGA)

teamStatsByRange <- allLogShort %>% group_by(TEAM_ABRV, RANGE) %>% summarise(FGM = sum(SHOT_MADE_FLAG),FGA = sum(SHOT_ATTEMPTED_FLAG),
                                                                     FG_PERCENTAGE = FGM/FGA, PPS =sum(SHOT_MADE_FLAG * SHOT_VALUE/FGA), PTS  = sum(SHOT_MADE_FLAG * SHOT_VALUE))
teamStats <- allLogShort %>% group_by(TEAM_ABRV) %>% summarise(FGM = sum(SHOT_MADE_FLAG),FGA = sum(SHOT_ATTEMPTED_FLAG),
                                                       PTS  = sum(SHOT_MADE_FLAG * SHOT_VALUE), EFG = PTS/(FGA*2), PPS = PTS/FGA)

Merging the range and total data sets for both players and teams we get the following data.


In [7]:
playerStatsByRange <- playerStatsByRange %>% merge(playerStats, by = 'PLAYER_NAME', suffixes = c("_range","_total"))
head(playerStatsByRange)

PLAYER_NAME,RANGE,FGM_range,FGA_range,FG_PERCENTAGE,PPS_range,PTS_range,FGM_total,FGA_total,PTS_total,EFG,PPS_total
Aaron Gordon,3PT,48,127,0.3779528,1.1338583,144,172,386,392,0.507772,1.015544
Aaron Gordon,LMR,14,48,0.2916667,0.5833333,28,172,386,392,0.507772,1.015544
Aaron Gordon,RIM,90,155,0.5806452,1.1612903,180,172,386,392,0.507772,1.015544
Aaron Gordon,SMR,20,56,0.3571429,0.7142857,40,172,386,392,0.507772,1.015544
Aaron Holiday,3PT,11,50,0.2200000,0.6600000,33,40,100,91,0.455000,0.910000
Aaron Holiday,LMR,10,17,0.5882353,1.1764706,20,40,100,91,0.455000,0.910000


In [8]:
teamStatsByRange <- teamStatsByRange %>% merge(teamStats, by = 'TEAM_ABRV', suffixes = c("_range","_total"))
head(teamStatsByRange)

TEAM_ABRV,RANGE,FGM_range,FGA_range,FG_PERCENTAGE,PPS_range,PTS_range,FGM_total,FGA_total,PTS_total,EFG,PPS_total
ATL,3PT,355,1082,0.3280961,0.9842884,1065,1228,2746,2812,0.5120175,1.024035
ATL,LMR,88,212,0.4150943,0.8349057,177,1228,2746,2812,0.5120175,1.024035
ATL,RIM,669,1124,0.5951957,1.1903915,1338,1228,2746,2812,0.5120175,1.024035
ATL,SMR,116,328,0.3536585,0.7073171,232,1228,2746,2812,0.5120175,1.024035
BKN,3PT,413,1144,0.3610140,1.0830420,1239,1344,2975,3102,0.5213445,1.042689
BKN,LMR,86,231,0.3722944,0.7489177,173,1344,2975,3102,0.5213445,1.042689


Calculate shotRangeRatio (% of shots at each range)

In [9]:
playerStatsByRange$shotRangeRatio <- playerStatsByRange$FGA_range/playerStatsByRange$FGA_total
teamStatsByRange$shotRangeRatio <- teamStatsByRange$FGA_range/teamStatsByRange$FGA_total

## Analysis

### Players by range sorted on shotRangeRatio (% of shots at each range)

In [10]:
rimPlayer <- playerStatsByRange %>% filter(RANGE == "RIM" & FGA_range >20) %>% arrange(desc(shotRangeRatio))

smrPlayer <- playerStatsByRange %>% filter(RANGE == "SMR" & FGA_range >20) %>% arrange(desc(shotRangeRatio))

lmrPlayer <- playerStatsByRange %>% filter(RANGE == "LMR" & FGA_range >20) %>% arrange(desc(shotRangeRatio))

threePlayer <- playerStatsByRange %>% filter(RANGE == "3PT" & FGA_range >20) %>% arrange(desc(shotRangeRatio))

head(rimPlayer, n=10L)
head(smrPlayer, n=10L)
head(lmrPlayer, n=10L)
head(threePlayer, n=10L)

PLAYER_NAME,RANGE,FGM_range,FGA_range,FG_PERCENTAGE,PPS_range,PTS_range,FGM_total,FGA_total,PTS_total,EFG,PPS_total,shotRangeRatio
Tyson Chandler,RIM,35,57,0.6140351,1.228070,70,35,59,70,0.5932203,1.186441,0.9661017
Mitchell Robinson,RIM,53,85,0.6235294,1.247059,106,56,88,112,0.6363636,1.272727,0.9659091
Ed Davis,RIM,77,123,0.6260163,1.252033,154,78,131,156,0.5954198,1.190840,0.9389313
Robert Williams III,RIM,18,22,0.8181818,1.636364,36,18,24,36,0.7500000,1.500000,0.9166667
Rudy Gobert,RIM,170,252,0.6746032,1.349206,340,180,275,360,0.6545455,1.309091,0.9163636
Damian Jones,RIM,48,66,0.7272727,1.454545,96,52,73,104,0.7123288,1.424658,0.9041096
DeAndre Jordan,RIM,120,183,0.6557377,1.311475,240,125,203,250,0.6157635,1.231527,0.9014778
Clint Capela,RIM,220,323,0.6811146,1.362229,440,231,362,462,0.6381215,1.276243,0.8922652
Cristiano Felicio,RIM,25,42,0.5952381,1.190476,50,27,49,54,0.5510204,1.102041,0.8571429
Joakim Noah,RIM,13,27,0.4814815,0.962963,26,15,32,30,0.4687500,0.937500,0.8437500


PLAYER_NAME,RANGE,FGM_range,FGA_range,FG_PERCENTAGE,PPS_range,PTS_range,FGM_total,FGA_total,PTS_total,EFG,PPS_total,shotRangeRatio
Kosta Koufos,SMR,14,38,0.3684211,0.7368421,28,33,73,66,0.4520548,0.9041096,0.5205479
Shaun Livingston,SMR,17,46,0.3695652,0.7391304,34,40,92,80,0.4347826,0.8695652,0.5000000
Robin Lopez,SMR,27,55,0.4909091,0.9818182,54,64,121,131,0.5413223,1.0826446,0.4545455
Harry Giles III,SMR,18,40,0.4500000,0.9000000,36,48,102,96,0.4705882,0.9411765,0.3921569
T.J. McConnell,SMR,27,49,0.5510204,1.1020408,54,73,129,152,0.5891473,1.1782946,0.3798450
Tyrone Wallace,SMR,16,28,0.5714286,1.1428571,32,36,80,72,0.4500000,0.9000000,0.3500000
DeMar DeRozan,SMR,98,209,0.4688995,0.9377990,196,289,603,584,0.4842454,0.9684909,0.3466003
Marcin Gortat,SMR,14,35,0.4000000,0.8000000,28,52,104,104,0.5000000,1.0000000,0.3365385
Jeff Teague,SMR,31,74,0.4189189,0.8378378,62,86,223,191,0.4282511,0.8565022,0.3318386
Jonas Valanciunas,SMR,43,84,0.5119048,1.0238095,86,149,258,307,0.5949612,1.1899225,0.3255814


PLAYER_NAME,RANGE,FGM_range,FGA_range,FG_PERCENTAGE,PPS_range,PTS_range,FGM_total,FGA_total,PTS_total,EFG,PPS_total,shotRangeRatio
Collin Sexton,LMR,81,189,0.4285714,0.8571429,162,201,456,427,0.4682018,0.9364035,0.4144737
Terrence Ross,LMR,68,138,0.4927536,0.9855072,136,158,357,380,0.5322129,1.0644258,0.3865546
Gorgui Dieng,LMR,22,54,0.4074074,0.8148148,44,68,147,141,0.4795918,0.9591837,0.3673469
Klay Thompson,LMR,105,217,0.4838710,0.9677419,210,281,632,645,0.5102848,1.0205696,0.3433544
Jamal Crawford,LMR,30,64,0.4687500,0.9375000,60,67,188,158,0.4202128,0.8404255,0.3404255
DeMar DeRozan,LMR,83,200,0.4150000,0.8300000,166,289,603,584,0.4842454,0.9684909,0.3316750
Darren Collison,LMR,35,80,0.4375000,0.8750000,70,107,249,241,0.4839357,0.9678715,0.3212851
JR Smith,LMR,9,25,0.3600000,0.7200000,18,27,78,66,0.4230769,0.8461538,0.3205128
LaMarcus Aldridge,LMR,64,158,0.4050633,0.8101266,128,242,502,486,0.4840637,0.9681275,0.3147410
Jose Calderon,LMR,11,22,0.5000000,1.0000000,22,24,70,54,0.3857143,0.7714286,0.3142857


PLAYER_NAME,RANGE,FGM_range,FGA_range,FG_PERCENTAGE,PPS_range,PTS_range,FGM_total,FGA_total,PTS_total,EFG,PPS_total,shotRangeRatio
Gary Clark,3PT,21,76,0.2763158,0.8289474,63,25,83,71,0.4277108,0.8554217,0.9156627
Anthony Tolliver,3PT,23,60,0.3833333,1.1500000,69,29,68,81,0.5955882,1.1911765,0.8823529
Wayne Ellington,3PT,45,125,0.3600000,1.0800000,135,53,150,151,0.5033333,1.0066667,0.8333333
Alex Abrines,3PT,39,119,0.3277311,0.9831933,117,52,146,143,0.4897260,0.9794521,0.8150685
Darius Miller,3PT,49,127,0.3858268,1.1574803,147,67,162,183,0.5648148,1.1296296,0.7839506
Troy Daniels,3PT,23,58,0.3965517,1.1896552,69,31,75,85,0.5666667,1.1333333,0.7733333
Gerald Green,3PT,46,132,0.3484848,1.0454545,138,71,171,188,0.5497076,1.0994152,0.7719298
Davis Bertans,3PT,52,107,0.4859813,1.4579439,156,73,147,198,0.6734694,1.3469388,0.7278912
PJ Tucker,3PT,63,152,0.4144737,1.2434211,189,91,210,245,0.5833333,1.1666667,0.7238095
Landry Shamet,3PT,62,149,0.4161074,1.2483221,186,92,206,246,0.5970874,1.1941748,0.7233010


### Players by PPS

In [11]:
rimPlayer <- playerStatsByRange %>% filter(RANGE == "RIM" & FGA_range >20) %>% arrange(desc(PPS_range))

smrPlayer <- playerStatsByRange %>% filter(RANGE == "SMR" & FGA_range >20) %>% arrange(desc(PPS_range))

lmrPlayer <- playerStatsByRange %>% filter(RANGE == "LMR" & FGA_range >20) %>% arrange(desc(PPS_range))

threePlayer <- playerStatsByRange %>% filter(RANGE == "3PT" & FGA_range >20) %>% arrange(desc(PPS_range))


head(rimPlayer, n=10L)
head(smrPlayer, n=10L)
head(lmrPlayer, n=10L)
head(threePlayer, n=10L)

PLAYER_NAME,RANGE,FGM_range,FGA_range,FG_PERCENTAGE,PPS_range,PTS_range,FGM_total,FGA_total,PTS_total,EFG,PPS_total,shotRangeRatio
Robert Williams III,RIM,18,22,0.8181818,1.636364,36,18,24,36,0.7500000,1.5000000,0.9166667
Jon Leuer,RIM,22,27,0.8148148,1.629630,44,33,53,66,0.6226415,1.2452830,0.5094340
Dwight Powell,RIM,69,85,0.8117647,1.623529,138,80,138,168,0.6086957,1.2173913,0.6159420
Justin Jackson,RIM,20,25,0.8000000,1.600000,40,80,177,193,0.5451977,1.0903955,0.1412429
Thomas Bryant,RIM,58,73,0.7945205,1.589041,116,68,103,141,0.6844660,1.3689320,0.7087379
Donte DiVincenzo,RIM,24,31,0.7741935,1.548387,48,41,106,98,0.4622642,0.9245283,0.2924528
Dwayne Bacon,RIM,17,22,0.7727273,1.545455,34,36,69,82,0.5942029,1.1884058,0.3188406
Serge Ibaka,RIM,101,131,0.7709924,1.541985,202,214,387,451,0.5826873,1.1653747,0.3385013
Giannis Antetokounmpo,RIM,250,325,0.7692308,1.538462,500,290,490,589,0.6010204,1.2020408,0.6632653
MarShon Brooks,RIM,35,46,0.7608696,1.521739,70,76,167,167,0.5000000,1.0000000,0.2754491


PLAYER_NAME,RANGE,FGM_range,FGA_range,FG_PERCENTAGE,PPS_range,PTS_range,FGM_total,FGA_total,PTS_total,EFG,PPS_total,shotRangeRatio
Otto Porter Jr.,SMR,21,36,0.5833333,1.166667,42,116,237,267,0.5632911,1.1265823,0.1518987
Tyrone Wallace,SMR,16,28,0.5714286,1.142857,32,36,80,72,0.4500000,0.9000000,0.3500000
T.J. McConnell,SMR,27,49,0.5510204,1.102041,54,73,129,152,0.5891473,1.1782946,0.3798450
Domantas Sabonis,SMR,34,64,0.5312500,1.062500,68,174,280,353,0.6303571,1.2607143,0.2285714
Jarrett Allen,SMR,17,32,0.5312500,1.062500,34,146,246,295,0.5995935,1.1991870,0.1300813
Kris Dunn,SMR,13,25,0.5200000,1.040000,26,40,84,83,0.4940476,0.9880952,0.2976190
Tomas Satoransky,SMR,13,25,0.5200000,1.040000,26,72,143,162,0.5664336,1.1328671,0.1748252
Tony Parker,SMR,27,52,0.5192308,1.038462,54,108,240,223,0.4645833,0.9291667,0.2166667
Rudy Gay,SMR,42,81,0.5185185,1.037037,84,163,305,361,0.5918033,1.1836066,0.2655738
Tristan Thompson,SMR,43,83,0.5180723,1.036145,86,141,255,282,0.5529412,1.1058824,0.3254902


PLAYER_NAME,RANGE,FGM_range,FGA_range,FG_PERCENTAGE,PPS_range,PTS_range,FGM_total,FGA_total,PTS_total,EFG,PPS_total,shotRangeRatio
Serge Ibaka,LMR,46,76,0.6052632,1.210526,92,214,387,451,0.5826873,1.1653747,0.1963824
Emmanuel Mudiay,LMR,27,46,0.5869565,1.195652,55,133,287,294,0.5121951,1.0243902,0.1602787
Darius Miller,LMR,14,24,0.5833333,1.166667,28,67,162,183,0.5648148,1.1296296,0.1481481
Jeremy Lin,LMR,14,24,0.5833333,1.166667,28,99,194,226,0.5824742,1.1649485,0.1237113
Kemba Walker,LMR,45,79,0.5696203,1.139241,90,259,613,618,0.5040783,1.0081566,0.1288744
Rudy Gay,LMR,35,62,0.5645161,1.129032,70,163,305,361,0.5918033,1.1836066,0.2032787
J.J. Barea,LMR,24,43,0.5581395,1.116279,48,128,291,283,0.4862543,0.9725086,0.1477663
Quinn Cook,LMR,29,53,0.5471698,1.094340,58,92,198,220,0.5555556,1.1111111,0.2676768
CJ McCollum,LMR,65,119,0.5462185,1.092437,130,260,559,581,0.5196780,1.0393560,0.2128801
Donovan Mitchell,LMR,30,55,0.5454545,1.090909,60,217,534,490,0.4588015,0.9176030,0.1029963


PLAYER_NAME,RANGE,FGM_range,FGA_range,FG_PERCENTAGE,PPS_range,PTS_range,FGM_total,FGA_total,PTS_total,EFG,PPS_total,shotRangeRatio
Willy Hernangomez,3PT,12,22,0.5454545,1.636364,36,59,109,130,0.5963303,1.192661,0.2018349
Dante Cunningham,3PT,25,49,0.5102041,1.530612,75,52,108,129,0.5972222,1.194444,0.4537037
Davis Bertans,3PT,52,107,0.4859813,1.457944,156,73,147,198,0.6734694,1.346939,0.7278912
Bojan Bogdanovic,3PT,70,145,0.4827586,1.448276,210,190,369,450,0.6097561,1.219512,0.3929539
Seth Curry,3PT,27,56,0.4821429,1.446429,81,42,109,111,0.5091743,1.018349,0.5137615
Joe Harris,3PT,74,156,0.4743590,1.423077,222,148,297,370,0.6228956,1.245791,0.5252525
Derrick Rose,3PT,45,95,0.4736842,1.421053,135,215,442,475,0.5373303,1.074661,0.2149321
Rudy Gay,3PT,35,75,0.4666667,1.400000,105,163,305,361,0.5918033,1.183607,0.2459016
Stephen Curry,3PT,111,238,0.4663866,1.399160,333,210,428,531,0.6203271,1.240654,0.5560748
Danilo Gallinari,3PT,75,164,0.4573171,1.371951,225,182,399,439,0.5501253,1.100251,0.4110276


### Team by shotRangeRatio for each range


In [12]:
rimTeam <- teamStatsByRange %>% filter(RANGE == "RIM") %>% arrange(desc(shotRangeRatio))
smrTeam <- teamStatsByRange %>% filter(RANGE == "SMR") %>% arrange(desc(shotRangeRatio))
lmrTeam <- teamStatsByRange %>% filter(RANGE == "LMR") %>%  arrange(desc(shotRangeRatio))
threeTeam <- teamStatsByRange %>% filter(RANGE == "3PT") %>% arrange(desc(shotRangeRatio))

head(rimTeam)
head(smrTeam)
head(lmrTeam)
head(threeTeam)

TEAM_ABRV,RANGE,FGM_range,FGA_range,FG_PERCENTAGE,PPS_range,PTS_range,FGM_total,FGA_total,PTS_total,EFG,PPS_total,shotRangeRatio
LAL,RIM,780,1223,0.6377760,1.275552,1560,1381,2889,3106,0.5375562,1.075112,0.4233299
ATL,RIM,669,1124,0.5951957,1.190391,1338,1228,2746,2812,0.5120175,1.024035,0.4093227
MIL,RIM,760,1134,0.6701940,1.340388,1520,1331,2795,3089,0.5525939,1.105188,0.4057245
OKC,RIM,684,1127,0.6069210,1.213842,1368,1301,2851,2911,0.5105226,1.021045,0.3952999
UTA,RIM,675,1084,0.6226937,1.245387,1350,1267,2756,2890,0.5243106,1.048621,0.3933237
NOP,RIM,741,1174,0.6311755,1.262351,1482,1426,2994,3186,0.5320641,1.064128,0.3921176


TEAM_ABRV,RANGE,FGM_range,FGA_range,FG_PERCENTAGE,PPS_range,PTS_range,FGM_total,FGA_total,PTS_total,EFG,PPS_total,shotRangeRatio
SAS,SMR,312,666,0.4684685,0.9369369,624,1381,2900,3078,0.5306897,1.061379,0.2296552
MEM,SMR,208,531,0.3917137,0.7834275,416,1185,2602,2676,0.5142198,1.028440,0.2040738
MIN,SMR,238,571,0.4168126,0.8336252,476,1294,2881,2913,0.5055536,1.011107,0.1981951
MIA,SMR,182,527,0.3453510,0.6907021,364,1183,2760,2735,0.4954710,0.990942,0.1909420
LAC,SMR,225,523,0.4302103,0.8604207,450,1315,2776,2924,0.5266571,1.053314,0.1884006
SAC,SMR,209,539,0.3877551,0.7755102,418,1381,2919,3129,0.5359712,1.071942,0.1846523


TEAM_ABRV,RANGE,FGM_range,FGA_range,FG_PERCENTAGE,PPS_range,PTS_range,FGM_total,FGA_total,PTS_total,EFG,PPS_total,shotRangeRatio
SAS,LMR,282,677,0.4165436,0.8330871,564,1381,2900,3078,0.5306897,1.0613793,0.2334483
GSW,LMR,306,630,0.4857143,0.9714286,612,1405,2900,3202,0.5520690,1.1041379,0.2172414
CLE,LMR,238,575,0.4139130,0.8278261,476,1282,2871,2857,0.4975618,0.9951237,0.2002786
IND,LMR,219,527,0.4155598,0.8311195,438,1339,2821,2983,0.5287132,1.0574264,0.1868132
ORL,LMR,202,501,0.4031936,0.8083832,405,1203,2717,2750,0.5060729,1.0121457,0.1843946
NYK,LMR,215,501,0.4291417,0.8602794,431,1313,2992,2961,0.4948195,0.9896390,0.1674465


TEAM_ABRV,RANGE,FGM_range,FGA_range,FG_PERCENTAGE,PPS_range,PTS_range,FGM_total,FGA_total,PTS_total,EFG,PPS_total,shotRangeRatio
HOU,3PT,444,1280,0.3468750,1.0406250,1332,1175,2607,2796,0.5362486,1.072497,0.4909858
MIL,3PT,427,1219,0.3502871,1.0508614,1281,1331,2795,3089,0.5525939,1.105188,0.4361360
BOS,3PT,408,1128,0.3617021,1.0851064,1224,1267,2806,2942,0.5242338,1.048468,0.4019957
ATL,3PT,355,1082,0.3280961,0.9842884,1065,1228,2746,2812,0.5120175,1.024035,0.3940277
DAL,3PT,360,1032,0.3488372,1.0465116,1080,1207,2641,2774,0.5251799,1.050360,0.3907611
BKN,3PT,413,1144,0.3610140,1.0830420,1239,1344,2975,3102,0.5213445,1.042689,0.3845378


### Team by PPS for each range


In [13]:
rimTeam <- teamStatsByRange %>% filter(RANGE == "RIM") %>% arrange(desc(PPS_range))#desc(shotRangeRatio), desc(EFG))
smrTeam <- teamStatsByRange %>% filter(RANGE == "SMR") %>% arrange(desc(PPS_range))#desc(shotRangeRatio), desc(EFG))
lmrTeam <- teamStatsByRange %>% filter(RANGE == "LMR") %>% arrange(desc(PPS_range))#shotRangeRatio), desc(EFG))
threeTeam <- teamStatsByRange %>% filter(RANGE == "3PT") %>% arrange(desc(PPS_range))#shotRangeRatio), desc(EFG))

head(rimTeam)
head(smrTeam)
head(lmrTeam)
head(threeTeam)

TEAM_ABRV,RANGE,FGM_range,FGA_range,FG_PERCENTAGE,PPS_range,PTS_range,FGM_total,FGA_total,PTS_total,EFG,PPS_total,shotRangeRatio
MIL,RIM,760,1134,0.6701940,1.340388,1520,1331,2795,3089,0.5525939,1.105188,0.4057245
GSW,RIM,515,789,0.6527250,1.305450,1030,1405,2900,3202,0.5520690,1.104138,0.2720690
PHI,RIM,624,967,0.6452947,1.290589,1248,1350,2887,3068,0.5313474,1.062695,0.3349498
TOR,RIM,702,1091,0.6434464,1.286893,1404,1464,3032,3315,0.5466689,1.093338,0.3598285
WAS,RIM,697,1090,0.6394495,1.278899,1394,1377,2942,3111,0.5287220,1.057444,0.3704963
LAL,RIM,780,1223,0.6377760,1.275552,1560,1381,2889,3106,0.5375562,1.075112,0.4233299


TEAM_ABRV,RANGE,FGM_range,FGA_range,FG_PERCENTAGE,PPS_range,PTS_range,FGM_total,FGA_total,PTS_total,EFG,PPS_total,shotRangeRatio
SAS,SMR,312,666,0.4684685,0.9369369,624,1381,2900,3078,0.5306897,1.061379,0.2296552
TOR,SMR,225,500,0.4500000,0.9000000,450,1464,3032,3315,0.5466689,1.093338,0.1649077
LAC,SMR,225,523,0.4302103,0.8604207,450,1315,2776,2924,0.5266571,1.053314,0.1884006
BOS,SMR,182,430,0.4232558,0.8465116,364,1267,2806,2942,0.5242338,1.048468,0.1532431
IND,SMR,181,429,0.4219114,0.8438228,362,1339,2821,2983,0.5287132,1.057426,0.1520737
DEN,SMR,200,475,0.4210526,0.8421053,400,1239,2662,2784,0.5229151,1.045830,0.1784373


TEAM_ABRV,RANGE,FGM_range,FGA_range,FG_PERCENTAGE,PPS_range,PTS_range,FGM_total,FGA_total,PTS_total,EFG,PPS_total,shotRangeRatio
GSW,LMR,306,630,0.4857143,0.9714286,612,1405,2900,3202,0.5520690,1.104138,0.21724138
TOR,LMR,150,313,0.4792332,0.9584665,300,1464,3032,3315,0.5466689,1.093338,0.10323219
DEN,LMR,120,272,0.4411765,0.8860294,241,1239,2662,2784,0.5229151,1.045830,0.10217881
HOU,LMR,40,93,0.4301075,0.8817204,82,1175,2607,2796,0.5362486,1.072497,0.03567319
POR,LMR,189,429,0.4405594,0.8811189,378,1296,2830,2941,0.5196113,1.039223,0.15159011
UTA,LMR,102,234,0.4358974,0.8717949,204,1267,2756,2890,0.5243106,1.048621,0.08490566


TEAM_ABRV,RANGE,FGM_range,FGA_range,FG_PERCENTAGE,PPS_range,PTS_range,FGM_total,FGA_total,PTS_total,EFG,PPS_total,shotRangeRatio
SAS,3PT,316,789,0.4005070,1.201521,948,1381,2900,3078,0.5306897,1.061379,0.2720690
SAC,3PT,367,956,0.3838912,1.151674,1101,1381,2919,3129,0.5359712,1.071942,0.3275094
GSW,3PT,392,1022,0.3835616,1.150685,1176,1405,2900,3202,0.5520690,1.104138,0.3524138
LAC,3PT,294,789,0.3726236,1.117871,882,1315,2776,2924,0.5266571,1.053314,0.2842219
IND,3PT,305,820,0.3719512,1.115854,915,1339,2821,2983,0.5287132,1.057426,0.2906771
BOS,3PT,408,1128,0.3617021,1.085106,1224,1267,2806,2942,0.5242338,1.048468,0.4019957


## NBA Summary stats

Look at teh PPS_range and shotRangeRatio to see averages in the NBA for each range.

In [14]:
allNBA <- teamStatsByRange %>% group_by(RANGE) %>% summarise(TEAM_ABRV = "All" ,FGA_range = sum(FGA_range), FGM_range = sum(FGM_range), 
                                                             FG_PERCENTAGE = FGM_range/FGA_range,PTS_range = sum(PTS_range), EFG = mean(EFG),
                                                             FGM_total = sum(.$FGM_range), FGA_total = sum(.$FGA_range), PTS_total= sum(.$PTS_range),
                                                             PPS_range = PTS_range/FGA_range,shotRangeRatio = mean(shotRangeRatio), 
                                                             PPS_total = sum(.$PTS_range)/sum(.$FGA_range))
head(allNBA)

RANGE,TEAM_ABRV,FGA_range,FGM_range,FG_PERCENTAGE,PTS_range,EFG,FGM_total,FGA_total,PTS_total,PPS_range,shotRangeRatio,PPS_total
3PT,All,29807,10498,0.3521991,31494,0.5213737,38939,84743,88385,1.0565974,0.3523731,1.042977
LMR,All,10903,4479,0.4108044,8967,0.5213737,38939,84743,88385,0.8224342,0.1280148,1.042977
RIM,All,30312,18483,0.6097585,36966,0.5213737,38939,84743,88385,1.2195170,0.3577424,1.042977
SMR,All,13721,5479,0.3993149,10958,0.5213737,38939,84743,88385,0.7986298,0.1618696,1.042977


## Wrap up and export to csv for Shiny app use

In [15]:
teamStatsByRange <- rbind(teamStatsByRange, allNBA)
write_csv(teamStatsByRange,"./catch-and-shoot/longTeamByRange.csv")
write_csv(allLogShort,"./catch-and-shoot/all-log-short.csv")